Pytorch에서 제공하는 RNN 클래스는 기본적으로 Elman RNN 이지만,
제공하는 클래스 대신 직접 Elman RNN을 구현한다.
하나의 RNN 타임 스텝을 구현한 RNNCell을 사용하여 Elman RNN을 구현해보자.

In [ ]:
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class ElmanRNN(nn.Module):
  def __init__(self, input_size, hidden_size, batch_first=False): # input_size : 입력 벡터 크기, hidden_size : 은닉 상태 벡터 크기, batch_first : 0번째 차원이 배치인지 여부
    super(ElmanRNN, self).__init__()
    self.rnn_cell = nn.RNNCell(input_size, hidden_size)

    self.batch_first = batch_first  # 0번째 차원이 배치인지 여부
    self.hidden_size = hidden_size  # 은닉 상태 벡터 크기

  def _initialize_hidden(self, batch_size):
    return torch.zeros((batch_size, self.hidden_size)) # 형상은 (N x H)가 되며, N : 배치 크기, H :은닉 상태 크기

  def forward(self, x_in, initial_hidden=None):
    '''
    0번째 차원이 배치라면 x_in의 0번째와 1번째 순서를 바꾸어준다.
    반대로 0번째 차원이 배치가 아닌 경우에는
    0번째 차원 : 시퀀스 크기 (시계열 데이터의 분량)
    1번째 차원 : 배치 크기
    '''
    if self.batch_first:
      batch_size, seq_size, feat_size = x_in.size()
      x_in = x_in.permute(1, 0, 2)
    else:
      seq_size, batch_size, feat_size = x_in.size()

    hiddens = []

    if initial_hidden is None:                    # 초기 은닉이 정해져있지 않다면, 초기화한다.
      initial_hidden = self._initialize_hidden(batch_size)
      initial_hidden = initial_hidden.to(x_in.device)

    hidden_t = initial_hidden

    for t in range(seq_size):
      hidden_t = self.rnn_cell(x_in[t], hidden_t) # 입력 벡터와 은닉 상태를 전달하는데, 출력된 은닉 상태는 다음의 rnn_cell에 전달된다.
      hiddens.append(hidden_t)

    hiddens = torch.stack(hiddens)

    if self.batch_first:                          # 0번째 차원이 배치라면 은닉 상태 순서를 바꿔준다.
      hiddens = hiddens.permute(1, 0, 2)

    return hiddens

문자 시퀀스를 입력으로 받아 성씨에 따른 국적 분류 예측을 해보자.
시퀀스는 다음과 같이 입력된다.
'Davidson' : 'D','a','v','i','d','s','o','n'

성씨 데이터셋 특징은 다음과 같다.
1. 데이터 속성이 불규칙 (영어 27%, 러시아어 21%, 아랍어 14%, 나머지 38%)
2. 출신 국가와 성씨 철자법 사이에 의미 있고 직관적 관계가 있다.


In [ ]:
import collections
import numpy as np
import pandas as pd
import re
from argparse import Namespace
from collections import Counter
import json
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

In [ ]:
args = Namespace(
    raw_dataset_csv="data/surnames/surnames.csv",
    train_proportion=0.7
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="data/surnames/surnames_with_splits.csv", # nationality_index, split 이 추가됨
    # 경로 정보
    surname_csv="data/surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/surname_classification",
    # 모델 하이퍼파라미터
    char_embedding_size=100,
    rnn_hidden_size=64,
    # 훈련 하이퍼파라미터
    seed=1337,
    num_epochs=100,
    learning_rate=1e-3,
    batch_size=64,
    early_stopping_criteria=5,
    # 실행 옵션
    cuda=True,
    expand_filepaths_to_save_dir=True
)

In [ ]:
class SurnameDataset(Dataset):
  def __init__(self, surname_df, vectorizer):
    """
    매개변수:
        surname_df (pandas.DataFrame): 데이터셋
        vectorizer (SurnameVectorizer): 데이터셋에서 만든 Vectorizer 객체
    """
    self.surname_df = surname_df
    self._vectorizer = vectorizer

    self._max_seq_length = max(map(len, self.surname_df.surname)) + 2 # 시퀀스의 최대 크기

    self.train_df = self.surname_df[self.surname_df.split=='train']
    self.train_size = len(self.train_df)

    self.val_df = self.surname_df[self.surname_df.split=='val']
    self.validation_size = len(self.val_df)

    self.test_df = self.surname_df[self.surname_df.split=='test']
    self.test_size = len(self.test_df)

    self._lookup_dict = {'train': (self.train_df, self.train_size),
                          'val': (self.val_df, self.validation_size),
                          'test': (self.test_df, self.test_size)}

    self.set_split('train')

    # 클래스 가중치
    class_counts = self.train_df.nationality.value_counts().to_dict()
    def sort_key(item):
        return self._vectorizer.nationality_vocab.lookup_token(item[0])
    sorted_counts = sorted(class_counts.items(), key=sort_key)
    frequencies = [count for _, count in sorted_counts]
    self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

  @classmethod
  def load_dataset_and_make_vectorizer(cls, surname_csv):
    surname_df = pd.read_csv(surname_csv)
    train_surname_df = surname_df[surname_df.split=='train']
    return cls(surname_df, SurnameVectorizer.from_dataframe(train_surname_df)) # SurnameDataset 객체 반환

  def __getitem__(self, index):
    row = self._target_df.iloc[index]

    surname_vector, vec_length = self._vectorizer.vectorize(row.surname, self._max_seq_length)

    nationality_index = self._vectorizer.nationality_vocab.lookup_token(row.nationality)

    return {'x_data':surname_vector,
            'y_target':nationality_index,
            'x_length':vec_length}

  def get_num_batches(self, batch_size): # 배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수
    return len(self) // batch_size


Vocabulary 클래스를 상속 받아 SequenceVocabulary 클래스를 선언하여 활용할 것이다.

SequenceVocabulary 클래스 역할
1. 주어진 이름 시퀀스의 각 "문자"를 정수로 매핑
2. 특별 토큰 활용


In [ ]:
class Vocabulary(object):
  """매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스 """

  def __init__(self, token_to_idx=None):
    """
    매개변수:
        token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
    """

    if token_to_idx is None:
        token_to_idx = {}
    self._token_to_idx = token_to_idx

    self._idx_to_token = {idx: token
                          for token, idx in self._token_to_idx.items()}

  def to_serializable(self):
    """ 직렬화할 수 있는 딕셔너리를 반환합니다 """
    return {'token_to_idx': self._token_to_idx}

  @classmethod
  def from_serializable(cls, contents):
    """ 직렬화된 딕셔너리에서 Vocabulary 객체를 만듭니다 """
    return cls(**contents)
  def add_token(self, token):
    """ 토큰을 기반으로 매핑 딕셔너리를 업데이트합니다

    매개변수:
        token (str): Vocabulary에 추가할 토큰
    반환값:
        index (int): 토큰에 상응하는 정수
    """
    if token in self._token_to_idx:
        index = self._token_to_idx[token]
    else:
        index = len(self._token_to_idx)
        self._token_to_idx[token] = index
        self._idx_to_token[index] = token
    return index

  def add_many(self, tokens):
    """토큰 리스트를 Vocabulary에 추가합니다.

    매개변수:
        tokens (list): 문자열 토큰 리스트
    반환값:
        indices (list): 토큰 리스트에 상응되는 인덱스 리스트
    """
    return [self.add_token(token) for token in tokens]

  def lookup_token(self, token):
    """토큰에 대응하는 인덱스를 추출합니다.

    매개변수:
        token (str): 찾을 토큰
    반환값:
        index (int): 토큰에 해당하는 인덱스
    """
    return self._token_to_idx[token]

  def lookup_index(self, index):
    """ 인덱스에 해당하는 토큰을 반환합니다.

    매개변수:
        index (int): 찾을 인덱스
    반환값:
        token (str): 인텍스에 해당하는 토큰
    에러:
        KeyError: 인덱스가 Vocabulary에 없을 때 발생합니다.
    """
    if index not in self._idx_to_token:
        raise KeyError("the index (%d) is not in the Vocabulary" % index)
    return self._idx_to_token[index]

  def __str__(self):
    return "<Vocabulary(size=%d)>" % len(self)

  def __len__(self):
    return len(self._token_to_idx)



'''
위에서 선언한 Vocabulary를 상속받아 SequenceVocabulary를 선언한다.
'''
class SequenceVocabulary(Vocabulary):
  def __init__(self, token_to_idx=None,
               unk_token="<UNK>",
               mask_token="<MASK>",
               begin_seq_token="<BEGIN>",
               end_seq_token="<END>"):
    super(SequenceVocabulary, self).__init__(token_to_idx)

    self._mask_token = mask_token
    self._unk_token = unk_token
    self._begin_seq_token = begin_seq_token
    self._end_seq_token = end_seq_token

    self.mask_index = self.add_token(self._mask_token)
    self.unk_index = self.add_token(self._unk_token)
    self.begin_seq_index = self.add_token(self._begin_seq_token)
    self.end_seq_index = self.add_token(self._end_seq_token)

  def to_serializable(self):
    contents = super(SequenceVocabulary, self).to_serializable()
    contents.update({'unk_token': self._unk_token,
                     'mask_token': self._mask_token,
                     'begin_seq_token': self._begin_seq_token,
                     'end_seq_token': self._end_seq_token})
    return contents

  def lookup_token(self, token):
    if self.unk_index >= 0: # UNK 토큰을 사용하려면 UNK 인덱스가 0보다 커야한다.
      # 만약 token에 해당하는 key를 찾을 수 없으면 디폴트로 unk_index를 반환한다.
      return self._token_to_idx.get(token, self.unk_index)
    else: # UNK 토큰을 사용하지 않을 경우..
      return self._token_to_idx[token]

SurnameVectorizer 클래스 선언

SurnameVectorizer 클래스 역할 : SequenceVocabulary를 사용해 성씨에 있는 문자와 정수 간의 매핑을 관리한다

In [ ]:
class SurnameVectorizer(object):
  '''어휘 사전을 생성하고 관리한다'''
  # surname 문자열, vector_length 인덱스 벡터 길이를 맞추기 위한 매개변수
  def vectorize(self, surname, vector_length=-1):
    '''입력으로 받은 성씨의 각 문자를 인덱스로 변환'''
    # BEGIN 토큰 인덱스 추가
    indices = [self.char_vocab.begin_seq_index]
    # 각 문자를 인덱스로 변환
    indices.extend(self.char_vocab, lookup_token(token) for token in surname)
    # END 토큰 인덱스 추가
    indices.append(self.char_vocab.end_seq_index)

    # 인덱스 벡터 길이가 안주어졌다면 벡터 길이를 문자들을 변환한 인덱스 길이로 초기화
    if vector_length < 0:
      vector_length = len(indices)

    '''
    시퀀스 길이 +2(특수 토큰 <BEGIN>, <END>를 추가한 indices 길이)의 넘파이 벡터를 생성하고
    각 위치의 인덱스 값을 저장
    '''
    out_vector = np.zeros(vector_length, dtype=np.int64)
    out_vector[:len(indices)] = indices
    # 입력 시퀀스가 vector_length보다 짧아 남는 공간은 MASK 인덱스로 패딩
    out_vector[len(indices):] = self.char_vocab.mask_index

    # 단어 인덱스를 저장한 벡터와 입력 시퀀스(시퀀스 길이+2) 길이를 반환
    return out_vector, len(indices)

  @classmethod
  def from_dataframe(cls, surname_df): # surname_df 성씨 데이터셋의 데이터프레임
    '''
    데이터셋 데이터프레임으로 SurnameVectorizer 객체를 초기화한다.
    '''
    char_vocab = SequenceVocabulary()
    nationality_vocab = Vocabulary()

    for index, row in surname_df.iterrows(): # 성씨 데이터셋의 행 순회
      for char in row.surname: # 성씨의 문자열 문자 순회
        char_vocab.add_token(char) # SequenceVocabulary 객체에 토큰으로 성씨 문자 추가
      nationality_vocab.add_token(row.nationality) # Vocabulary 객체에 토큰으로 nationality 추가

    return cls(char_vocab, nationality_vocab)

SurnameClassifier 클래스 선언


1. 임베딩 층을 통해 문자에 대한 정수들(모델 입력은 SequenceVocabulary에서 정수로 매핑한 토큰)을 임베딩한다.
2. RNN으로 시퀀스의 벡터 표현(은닉 상태)을 계산하고, RNN을 반복적으로 거쳐 성씨를 요약한 벡터를 추출한다.
3. 이 요약한 최종 벡터를 Linear층으로 전달하여 예측 벡터를 계산한다.
4. 이 예측 벡터를 사용하여 손실을 계산하거나 Softmax 활성 함수에 적용하여 성씨에 대한 확률 분포를 만든다.

In [ ]:
def column_gather(y_out, x_lengths):
  # TODO : y_out에 있는 각 데이터 포인트에서 마지막 벡터를 추출

class SurnameClassifier(nn.Module):
  def __init__(self, embedding_size, num_embeddings, num_classes, rnn_hidden_size, batch_first=True, padding_idx=0)
  '''
  매개 변수로 임베딩 크기, 임베딩 개수, 클래스 개수(예측 벡터의 크기, 국적 개수), RNN의 은닉 상태 크기가 필요
  '''
  super(SurnameClassifier, self).__init__()

  self.emb = nn.Embedding(num_embeddings=num_embeddings,
                          embedding_dim=embedding_size,
                          padding_idx=padding_idx)
  self.rnn = EmanRNN(input_size=embedding_size,
                     hidden_size=rnn_hidden_size,
                     batch_first=batch_first)
  # affine 계층
  self.fc1 = nn.Linear(in_features=rnn_hidden_size, out_features=rnn_hidden_size)
  self.fc2 = nn.Linear(in_features=rnn_hidden_size, out_features=num_classes)

  def forward(self, x_in, x_lengths=None, apply_softmax=False):
    x_embedded = self.emb(x_in)
    y_out = self.rnn(x_embedded)

    if x_lengths is not None:
      # x_lengths를 이용하여 rnn 계층을 통과한 마지막 출력 벡터를 구한다
      y_out = column_gather(y_out, x_lengths) # column_gather은 각 시퀀스의 최종 출력 벡터를 추출하는 역할의 함수다.
    else:
      y_out = y_out[:, -1, :]

    y_out = F.dropout(y_out, 0.5)
    y_out = F.relu(self.fc1(y_out))
    y_out = F.dropout(y_out, 0.5)
    y_out = F.fc2(y_out)
    if apply_softmax: # Softmax 활성화 함수를 적용할 경우 (크로스 엔트로피 손실을 사용하려면 False로 지정됨)
      y_out = F.softmax(y_out, dim=1)

    return y_out

In [ ]:
# TODO : 모델 훈련 및 평가